### Import all libraries

In [1]:
import face_recognition as fr
import cv2
import glob
import numpy as np
import os
import mediapipe as mp
import random
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier
import boto3
import botocore 
from PIL import Image
from matplotlib.pyplot import imshow
import pandas as pd
from io import StringIO
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
import pickle

In [5]:
"""
...
...
Run this cell to load training data
...
...
"""
data = "./own/*/"
x = []
y = []
classes = {}
to_add = {"id": [], "name": [], "insta": []}
#bucket_name = 'cnfacerecog'
#bucket = res.Bucket(bucket_name)
#keys_in_bucket = [o.key for o in bucket.objects.all()]
temp = 0
    
#num_classes = len(glob.glob(os.path.join(data)))
#n = random.randint(0, num_classes)
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    if idx == 3000:
        break
    num_pics = len(glob.glob(os.path.join(folder+"*")))
    if num_pics < 10:
        continue
    use = glob.glob(os.path.join(folder+"*"))[:10]
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    classes[idx] = person_name
    to_add["id"].append(idx)
    to_add["name"].append(person_name)
    to_add["insta"].append("")
    for f in use:
        """
        file_name = f.split("\\")[-1]
        bucket_file_name = "faces_5/"+str(idx)+"/"+file_name 
        if bucket_file_name in keys_in_bucket:
            continue
        response = s3.upload_file(f, bucket_name, bucket_file_name)"""
        
        pic = cv2.imread(f)
        detected_pos = fr.face_locations(pic)
        for i in detected_pos:
            top, right, bottom, left = i
            face_image = pic[top:bottom, left:right]
            try:
                face_image = cv2.resize(face_image, (256, 256))
                face_embedding = fr.face_encodings(face_image)[0]
                x.append(face_embedding)
                y.append(idx)
            except:
                continue


0 David_Schwimmer


In [6]:
print(len(x))

8


In [7]:
def save_dataset(l, name):
    with open("./saved/"+name, "wb") as fp:   #Pickling
        pickle.dump(l, fp)
        
def load_dataset(name):
    with open("./saved/"+name, "rb") as fp:   # Unpickling
        b = pickle.load(fp)
    return b;


In [8]:
saved_x = load_dataset("3000_10")
saved_y = load_dataset("3000_10_y")
for i in range(0, len(y)):
    y[i] += saved_y[-1]+1
    saved_y.append(y[i])
for i in x:
    saved_x.append(i)

In [9]:
print(len(saved_x), len(saved_y))
classes["2941"] = "David_Schwimmer"

879 879


In [57]:
#IGNORE THIS CELL

def updating_csv(df2):
    obj = s3.get_object(Bucket="cnfacerecog", Key="details.csv")["Body"]
    df = pd.read_csv(obj)
    df['insta'] = df['insta'].fillna("")
    if not(df.equals(df2)):
        df = df.append(df2, ignore_index = True)
        csv_buffer = StringIO()
        df.to_csv(csv_buffer, index=False)
        #res.Object("cnfacerecog", 'details.csv').put(Body=csv_buffer.getvalue())
    else:
        print("Data not changed")
df2 = pd.DataFrame(to_add)
updating_csv(df2)

Data not changed


In [4]:
"""



NOTE: PLSEASE IGNORE THIS CELL




x = []
y = []
bucket_name = 'cnfacerecog'
bucket = res.Bucket(bucket_name)
temp = 0
for o in bucket.objects.all():
    if o.key.find("faces_5/") != -1:
        idx = int(o.key.split("/")[1])
        if temp != idx:
            temp = idx
            print(idx)
        pic = s3.get_object(Bucket=bucket_name, Key=o.key) 
        pic = np.array(Image.open(pic["Body"]))
        detected_pos = fr.face_locations(pic)
        for i in detected_pos:
            top, right, bottom, left = i
            face_image = pic[top:bottom, left:right]
            try:
                face_image = cv2.resize(face_image, (256, 256))
                face_embedding = fr.face_encodings(face_image)[0]
                x.append(face_embedding)
                y.append(idx)
            except:
                continue
print(len(x))
print(len(y))"""

1007
1013
1021
1027
1033
1043
1045
1047
1059
107
1072
1091
1204
1207
121
1237
1252
1268
1270
1306
1309
1314
1333
1347
1382
1393
1473
1476
1480
1492
1536
1540
1558
1559
1565
1581
1589
1617
1695
1696
17
1704
1758
1784
1835
1843
1849
1854
1856
1862
1865
1870
189
1891
1894
1911
1932
1936
1940
1945
1962
1964
1966
197
1993
1998
204
210
213
222
223
239
241
272
290
297
300
302
317
335
353
358
370
374
380
384
386
401
417
471
479
481
50
502
506
527
544
550
551
552
562
573
579
58
604
610
619
627
642
706
73
736
744
747
760
781
783
802
816
827
831
843
847
852
875
882
89
896
904
923
942
946
97
673
673


In [6]:

#PLEASE IGNORE THIS CELL


def get_class_ids_from_bucket():
    bucket_name = 'cnfacerecog'
    bucket = res.Bucket(bucket_name)
    try:
        classes_in_bucket = list(set([o.key.split("/")[1] for o in bucket.objects.all() if o.key != "details.csv"]))
        classes_in_bucket.remove("")
        return classes_in_bucket
    except Exception as e:
        for o in bucket.objects.all():
            print(o.key)
    

classes_in_bucket = get_class_ids_from_bucket()
print(classes_in_bucket)

['1581', '1473', '2885', '2283', '2173', '370', '1059', '783', '1911', '875', '579', '1932', '58', '627', '374', '2419', '1382', '272', '1945', '831', '544', '2726', '2017', '1268', '197', '2264', '1091', '239', '471', '552', '1045', '2504', '1027', '1558', '1936', '1270', '1835', '802', '213', '2327', '1998', '1784', '2926', '97', '1862', '1207', '1565', '1617', '1894', '843', '2275', '1589', '506', '747', '223', '335', '1033', '2186', '2433', '1964', '2619', '2693', '2730', '2302', '2470', '2755', '760', '816', '550', '1306', '1047', '1309', '386', '2026', '1966', '2751', '297', '73', '2473', '1856', '2043', '2565', '610', '1865', '384', '2285', '2450', '1870', '2915', '551', '481', '189', '2526', '2500', '1204', '604', '401', '1843', '2857', '1072', '852', '1962', '2824', '1476', '573', '479', '317', '121', '210', '744', '2508', '1013', '1237', '736', '1021', '1347', '904', '1540', '1695', '1891', '2199', '1704', '706', '1559', '300', '1333', '527', '619', '946', '204', '1480', '847

In [55]:
import json 
with open("classes.json", "w") as outfile:
    json.dump(classes, outfile)

In [10]:
import json
class_f = open('classes.json')
classes = json.load(class_f)
print(dict(classes))

{'17': 'Abdullah_Gul', '50': 'Adrien_Brody', '121': 'Alejandro_Toledo', '204': 'Alvaro_Uribe', '223': 'Amelie_Mauresmo', '272': 'Andre_Agassi', '290': 'Andy_Roddick', '302': 'Angelina_Jolie', '317': 'Anna_Kournikova', '335': 'Ann_Veneman', '370': 'Ariel_Sharon', '374': 'Ari_Fleischer', '386': 'Arnold_Schwarzenegger', '417': 'Atal_Bihari_Vajpayee', '544': 'Bill_Clinton', '551': 'Bill_Gates', '562': 'Bill_McBride', '573': 'Bill_Simon', '706': 'Britney_Spears', '781': 'Carlos_Menem', '783': 'Carlos_Moya', '831': 'Catherine_Zeta-Jones', '875': 'Charles_Moose', '1047': 'Colin_Powell', '1059': 'Condoleezza_Rice', '1207': 'David_Beckham', '1252': 'David_Nalbandian', '1347': 'Dick_Cheney', '1382': 'Dominique_de_Villepin', '1393': 'Donald_Rumsfeld', '1473': 'Edmund_Stoiber', '1476': 'Eduardo_Duhalde', '1704': 'Fidel_Castro', '1854': 'George_HW_Bush', '1865': 'George_Robertson', '1870': 'George_W_Bush', '1891': 'Gerhard_Schroeder', '1932': 'Gloria_Macapagal_Arroyo', '1940': 'Gonzalo_Sanchez_de_L

In [59]:
model = OneVsRestClassifier(SVC(kernel='linear', probability=True, verbose=True, max_iter=3000))
#model = OneVsOneClassifier(SVC(kernel='linear', probability=True, verbose=True, max_iter=3000))
model.fit(saved_x, saved_y)
pickle.dump(model, open("./3000_10", 'wb'))

[LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM][LibSVM]

In [11]:
model = pickle.load(open("./3000_10",'rb'))

In [12]:
model.classes_

array([  17,   50,  121,  204,  223,  272,  290,  302,  317,  335,  370,
        374,  386,  417,  544,  551,  562,  573,  706,  781,  783,  831,
        875, 1047, 1059, 1207, 1252, 1347, 1382, 1393, 1473, 1476, 1704,
       1854, 1865, 1870, 1891, 1932, 1940, 1945, 1964, 1993, 2017, 2026,
       2043, 2055, 2131, 2155, 2173, 2186, 2199, 2210, 2264, 2275, 2283,
       2285, 2302, 2327, 2419, 2433, 2450, 2469, 2470, 2473, 2500, 2501,
       2504, 2508, 2526, 2565, 2619, 2661, 2685, 2687, 2693, 2726, 2730,
       2751, 2755, 2773, 2824, 2857, 2885, 2915, 2926, 2940, 2941])

In [13]:
class_list = list(classes.keys())
print(len(class_list))
print(class_list)

87
['17', '50', '121', '204', '223', '272', '290', '302', '317', '335', '370', '374', '386', '417', '544', '551', '562', '573', '706', '781', '783', '831', '875', '1047', '1059', '1207', '1252', '1347', '1382', '1393', '1473', '1476', '1704', '1854', '1865', '1870', '1891', '1932', '1940', '1945', '1964', '1993', '2017', '2026', '2043', '2055', '2131', '2155', '2173', '2186', '2199', '2210', '2264', '2275', '2283', '2285', '2302', '2327', '2419', '2433', '2450', '2469', '2470', '2473', '2500', '2501', '2504', '2508', '2526', '2565', '2619', '2661', '2685', '2687', '2693', '2726', '2730', '2751', '2755', '2773', '2824', '2857', '2885', '2915', '2926', '2940', '2941']


### Here we go one by one through each test face image

In [65]:
data = "./archive/group/*/"
#data = "../lfw/*/"
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    use = glob.glob(os.path.join(folder+"*"))
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    #classes[idx] = person_name
    for f in use:
        pic = cv2.imread(f)
        try:
            face_embedding = fr.face_encodings(pic)[0]
            test = np.asarray(face_embedding)
            test=  np.reshape(test, (1, -1))
            pred = model.predict_proba(test)[0]
            if np.max(pred) > 0.7:
                print(classes[str(class_list[np.argmax(pred)])], np.max(pred))
            elif np.max(pred) > 0.5 :
                #added to see what it predicts when highest probability is quite low
                #print(classes[str(class_list[np.argmax(pred)])], np.max(pred)) 
                choose_folder = "./lfw\\"+classes[str(class_list[np.argmax(pred)])]+"\\*"
                test_pic_path = glob.glob(os.path.join(choose_folder))[0]
                test_pic = cv2.imread(test_pic_path)
                try:
                    test_face_embedding = fr.face_encodings(test_pic)[0]
                    results = fr.compare_faces([face_embedding], test_face_embedding)
                    if results[0]:
                        print(classes[str(class_list[np.argmax(pred)])], np.max(pred)) 
                    else:
                        print("Face not found!")
                except Exception as first_e:
                    raise(first_e)
            else:
                print("Face not found!")
                    
        except Exception as e:
            raise(e)
            continue
            

0 lol
Bill_Clinton 0.8673548109600338
Face not found!
Face not found!
David_Schwimmer 0.8041707437431614
Jennifer_Aniston 0.8693508829717437


### Here we input all face embeddings in a single array - gives faster results

In [20]:
data = "./archive/group/*/"
testx = []
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    use = glob.glob(os.path.join(folder+"*"))
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    for f in use:
        pic = cv2.imread(f)
        try:
            all_emb = fr.face_encodings(pic)
            for test in all_emb:
                test = np.asarray(test)
                testx.append(test)
        except:
            continue
print(len(testx))

0 lol
19


In [21]:
"""probs = model.predict(testx)
for i in probs:
    print(classes[str(i)])"""

'probs = model.predict(testx)\nfor i in probs:\n    print(classes[str(i)])'

In [28]:
probs = model.predict_proba(testx)
present = []
for idx, pred in enumerate(probs):
    if np.max(pred) > 0.7:
        print(classes[(class_list[np.argmax(pred)])], np.max(pred))
        present.append(class_list[np.argmax(pred)])
    elif np.max(pred) > 0.5:
        choose_folder = "./lfw\\"+classes[(class_list[np.argmax(pred)])]+"\\*"
        real_face_embedding = testx[idx]
        test_pic_path = glob.glob(os.path.join(choose_folder))[0]
        test_pic = cv2.imread(test_pic_path)
        try:
            test_face_embedding = fr.face_encodings(test_pic)[0]
            results = fr.compare_faces([real_face_embedding], test_face_embedding)
            if results[0]:
                print(classes[(class_list[np.argmax(pred)])], np.max(pred)) 
                present.append(class_list[np.argmax(pred)])
            else:
                print("1. Face not found!")
                #print(classes[(class_list[np.argmax(pred)])], np.max(pred)) 
        except Exception as first_e:
            print("2. Face not found!")
            #print(classes[(class_list[np.argmax(pred)])], np.max(pred)) 
    else:
        print("3. Face not found!")
        #print(classes[(class_list[np.argmax(pred)])], np.max(pred)) 

Bill_Clinton 0.8673548109600336
Bill_Gates 0.724431984996856
3. Face not found!
Jennifer_Aniston 0.5778182425252121
3. Face not found!
3. Face not found!
David_Schwimmer 0.6541527634603629
3. Face not found!
3. Face not found!
1. Face not found!
Jennifer_Aniston 0.865090075415614
3. Face not found!
1. Face not found!
David_Schwimmer 0.7964122744891994
David_Schwimmer 0.8041707437431614
Jennifer_Aniston 0.871849494529098
Jennifer_Aniston 0.8693508829717439
Angelina_Jolie 0.8599382568050015
Jennifer_Lopez 0.7597069296295129


In [29]:
present = list(set(present))


['2508', '2941', '302', '544', '551', '2500']

In [30]:
for i in present:
    print(classes[i])

Jennifer_Lopez
David_Schwimmer
Angelina_Jolie
Bill_Clinton
Bill_Gates
Jennifer_Aniston


In [31]:
# 1 MEANS PRESENT, 0 MEANS ABSENT
for i in classes:
    if i in present:
        print(classes[i] + " 1")
    else:
        print(classes[i] + " 0")


Abdullah_Gul 0
Adrien_Brody 0
Alejandro_Toledo 0
Alvaro_Uribe 0
Amelie_Mauresmo 0
Andre_Agassi 0
Andy_Roddick 0
Angelina_Jolie 1
Anna_Kournikova 0
Ann_Veneman 0
Ariel_Sharon 0
Ari_Fleischer 0
Arnold_Schwarzenegger 0
Atal_Bihari_Vajpayee 0
Bill_Clinton 1
Bill_Gates 1
Bill_McBride 0
Bill_Simon 0
Britney_Spears 0
Carlos_Menem 0
Carlos_Moya 0
Catherine_Zeta-Jones 0
Charles_Moose 0
Colin_Powell 0
Condoleezza_Rice 0
David_Beckham 0
David_Nalbandian 0
Dick_Cheney 0
Dominique_de_Villepin 0
Donald_Rumsfeld 0
Edmund_Stoiber 0
Eduardo_Duhalde 0
Fidel_Castro 0
George_HW_Bush 0
George_Robertson 0
George_W_Bush 0
Gerhard_Schroeder 0
Gloria_Macapagal_Arroyo 0
Gonzalo_Sanchez_de_Lozada 0
Gordon_Brown 0
Gray_Davis 0
Guillermo_Coria 0
Halle_Berry 0
Hamid_Karzai 0
Hans_Blix 0
Harrison_Ford 0
Hillary_Clinton 0
Howard_Dean 0
Hugo_Chavez 0
Hu_Jintao 0
Ian_Thorpe 0
Igor_Ivanov 0
Jackie_Chan 0
Jack_Straw 0
Jacques_Chirac 0
Jacques_Rogge 0
James_Blake 0
James_Kelly 0
Jason_Kidd 0
Javier_Solana 0
Jean-David_Lev